In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics as stats
import arviz as az

In [2]:
flows = pd.read_csv(r'..\..\UK-wood-mfa\build\flows.csv')
flows_obs= pd.read_csv(r'..\..\UK-wood-mfa\build\flows_obs.csv')
Observations = pd.read_excel(r'..\..\UK-wood-mfa\data\Observations_with_references.xlsx')
flows_name = pd.read_excel(r'..\..\UK-wood-mfa\lists\flows_name.xlsx')

In [3]:
flows_name = list(flows_name.to_records())
flows_obs_list = list(flows_obs.to_records())

In [4]:
link = 'http://ukfires.org/analyses/UK-wood/system/'
Consumption = 'Consumption'

In [5]:
flows["flow_id"] = flows['target'].astype(str) +"-"+ flows["source"]+"-"+ flows["material"]
flow= flows["flow_id"].tolist()
flow_2 = list(dict.fromkeys(flow))
print(len(flow_2))

371


In [6]:
#This notebook is comparing observed values to the reconciled (calculated) ones. 
#It extracts the observed values for each monte-carlo run from flows_obs and matches these values with the results extracted 
#from flows. 

calculated_= []
observed_ = []
flows_analysis = []
t = 0
s= 0
flows_analysis = []
for flow in flows_name:
            calculated_2 = flows.loc[(flows['material'] == flow[3]) 
                                       & (flows['source'] == flow[1])
                                       & (flows['target'] == flow[2])]
            
            
            
            Source= 'Source:' + flow[1].replace('http://ukfires.org/analyses/UK-wood/system/', '')
            Target= ' Target:' + flow[2].replace('http://ukfires.org/analyses/UK-wood/system/', '') 
            Material= ' Material:' + flow[3].replace('http://ukfires.org/analyses/UK-wood/system/', '')
        
           
            
            
            observed_2 = flows_obs.loc[((flows_obs['role'] == 'ProcessInputObs') & ((flows_obs['object']) == flow[1]) & ((flows_obs['process']) == flow[2]))
                                           #   | ((flows_obs['role'] == 'ProcessOutput') & ((link + Observations['Process']) == flow[1]) & ((link + Observations['Object']) == flow[2]))
                                              | ((flows_obs['role'] == 'ImportsObs') & (('imports ' + flows_obs['object']) == flow[1]))
                                              | ((flows_obs['role'] == 'ExportsObs') & (('exports ' +  flows_obs['object']) == flow[2]))
                                               | ((flows_obs['role'] == 'ProductionObs') & ((flows_obs['object'] + 'Manufacturing') == flow[1]) & ((flows_obs['object']) == flow[2]))
                                               | ((flows_obs['role'] == 'ProductionObs')  & ((flows_obs['object']) == flow[2]))
                                                | ((flows_obs['role'] == 'ProductionObs') & (('SawmillsSoftwood') == flow[1]) & ((flows_obs['object']) == flow[2]))]
                                           #   | ((flows_obs['role'] == 'ProductionObs') & ((flows_obs['object']) == flow[2]))]
            
            
            #Append the result of each iteration to the list
    
            #print(observed_2)   
            
            for i in range(10):
                   
                    calculated_flow = calculated_2[calculated_2['sample']== i]
                    
                    observed_flow = observed_2[observed_2['solution']== i]
                    
                    calculated_flow_value_ = calculated_flow['value']
                    calculated_flow_value = calculated_flow_value_.tolist()

              

                    observed_flow_value_ = observed_flow['sampled_value']
                    observed_flow_value = observed_flow_value_.to_list()
                   

                   # print(observed_flow_value)

                 
                    #hdi_diff= hdiupper-hdilower

                    if calculated_flow_value:
                        calculated_value= calculated_flow_value[0]
                        
                    else:
                        calculated_value = 0
                        discrepancy_index = 0

                    if observed_flow_value:
                        observed_value= observed_flow_value[0]
                    else:
                        observed_value = 0
                        discrepancy_index = 0
                    
                        
                    if observed_value != 0 :
                        difference = calculated_value-observed_value
                    else:
                        difference = 0
                        
                    if observed_value != 0:
                        comparison = {'Source': Source ,
                                      'Target': Target,
                                      'Material': Material,
                                       'Observations': observed_value ,
                                       'Calculated': calculated_value,
                                       'Difference' : abs(difference) ,
                                      'Difference_relative' : abs(difference/observed_value) ,
                                        }

                        flows_analysis.append(comparison)


comparison2 = pd.DataFrame(flows_analysis, columns=["Source", "Target", "Material", "Observations" , "Calculated", "Difference", "Difference_relative"])
comparison2.to_excel(r"..\..\UK-wood-mfa\data_analysis\comparison_obs_results.xlsx")
            
        

                    


           


In [7]:


# Create a DataFrame from the sample data
df = pd.DataFrame(flows_analysis, columns=["Source", "Target", "Material", "Observations" , "Calculated", "Difference", "Difference_relative"])



# Group the DataFrame by the specified columns (Source, Target, Material)
grouped = df.groupby(["Source", "Target", "Material"])




# Calculate the mean difference for each group
min_diff = grouped["Difference"].min().reset_index()
max_diff = grouped["Difference"].max().reset_index()
mean_diff = grouped["Difference"].mean().reset_index()
min_obs = grouped["Observations"].min().reset_index()
max_obs = grouped["Observations"].max().reset_index()
mean_obs = grouped["Observations"].mean().reset_index()
min_calc = grouped["Calculated"].min().reset_index()
max_calc = grouped["Calculated"].max().reset_index()
mean_calc = grouped["Calculated"].mean().reset_index()
mean_relative_diff = grouped["Difference_relative"].mean().reset_index()
min_relative_diff = grouped["Difference_relative"].min().reset_index()
max_relative_diff = grouped["Difference_relative"].max().reset_index()


result_df = pd.DataFrame({
    'Source': min_diff['Source'],
    'Target': min_diff['Target'],
    'Material': min_diff['Material'],
    'Min_obs': min_obs['Observations'],
    'Max_obs': max_obs['Observations'],
    'Mean_obs': mean_obs['Observations'],
    'Min_calc': min_calc['Calculated'],
    'Max_calc': max_calc['Calculated'],
    'Mean_calc': mean_calc['Calculated'],
    'Min_Difference': min_diff['Difference'],
    'Max_Difference': max_diff['Difference'],
    'Mean_Difference': mean_diff['Difference'],
    'Min_Difference_relative': min_relative_diff['Difference_relative'],
    'Max_Difference_relative': max_relative_diff['Difference_relative'],
    'Mean_Difference_relative': mean_relative_diff['Difference_relative'],
})



# Display the result DataFrame
#print(result_df)
result_df.to_excel(r"..\..\UK-wood-mfa\data_analysis\comparison_obs_results_10.xlsx")
df.to_excel(r"..\..\UK-wood-mfa\data_analysis\comparison_obs_results_without_differences.xlsx")
